In [1]:
import numpy as np
import pandas as pd

from kcm.koopman_category_model import KoopmanCategoryModel

In [ ]:
eig_cols = [col for col in KCM.df_train if 'eig' in col]
norm_mode_cols = [col for col in KCM.df_train if 'norm_mode' in col]
KCM.df_train.loc[0,eig_cols]

In [ ]:
KCM.df_train.loc[0,norm_mode_cols]

In [ ]:
def filter_significant_modes(eigs, mode_norms, time_horizon=20, min_energy_frac=0.01):

    eigs = np.array(eigs)
    mode_norms = np.real(mode_norms)  # discard any small imaginary component
    assert np.all(mode_norms >= 0), "Mode norms must be non-negative"

    magnitudes = np.abs(eigs)

    # Avoid divide-by-zero for real eigenvalues near 1
    eps = 1e-8
    stable_mask = magnitudes < 1 - eps
    unstable_mask = magnitudes >= 1 - eps

    # Compute cumulative contribution over time
    decay_weight = np.zeros_like(magnitudes)

    # Stable (decaying) modes: geometric sum
    decay_weight[stable_mask] = (1 - magnitudes[stable_mask] ** (time_horizon + 1)) / \
                                 (1 - magnitudes[stable_mask])

    # Nearly persistent modes: just approximate as time_horizon + 1
    decay_weight[unstable_mask] = time_horizon + 1

    # Effective energy = norm × time contribution
    effective_energy = mode_norms * decay_weight

    # Normalize
    total_energy = effective_energy.sum()
    energy_frac = effective_energy / total_energy

    # Keep modes that contribute enough
    keep_mask = energy_frac > min_energy_frac

    return keep_mask

kept = np.array([filter_significant_modes(KCM.df_train.loc[i,eig_cols], KCM.df_train.loc[i,norm_mode_cols], time_horizon=20, min_energy_frac=0.01).sum() for i in range(KCM.df_train.shape[0])])